In [2]:
import pandas as pd
from collections import defaultdict
import mysql.connector
import glob
import os
import csv
import numpy as np
pd.set_option('display.max_columns', 10)

In [3]:
conn = mysql.connector.connect(user='root', password='password', host='127.0.0.1', port='3306',
                               database='expression_atlas_cells')
mycursor = conn.cursor(buffered=True)

# check the connection
if conn.is_connected():
    print("connection succesfull")
else:
    print("no connection")

connection succesfull


1. Only projects with confirmed tissue annotation will be accessible in the database. These PXDs can be found in the file manual_tissue_annotation.csv.

In [3]:
manual_meta = pd.read_csv('Metadata/manual_tissue_annotation_shotgun_checked_22092021.csv', sep=';')
manual_meta = manual_meta.fillna('Unknown')
projects = list(manual_meta['accession'].unique())
print(len(projects))

217


2. The metadata file from the PRIDE reprocessing (acquired via Niels) will be used to populate the **'project'** table <br>

project_id: automatically generated <br>
PXD_accession <br>
experiment_type <br>
instrument <br>
keywords <br>


In [11]:
meta = pd.read_csv('Metadata/metadata_human_shotgun.csv', sep='\t', encoding='utf-8')
meta = meta.astype(str)

In [12]:
def build_project_table(meta_df, list_of_pxds):
    count = 0
    meta_df = meta_df[meta_df['accession'].isin(list_of_pxds)]
    meta_df = meta_df[['accession', 'experimentTypes', 'instrumentNames', 'keywords', 'references']]
    meta_df = meta_df.astype(str)
    meta_tuples = list(meta_df.to_records(index=False)) #a list of tuples is easily iteratible and easy to store in the database
    for i in meta_tuples:
        count += 1
        project = "INSERT INTO project(PXD_accession, experiment_type, instrument, keywords, ref) VALUES (%s, %s, %s, %s, %s)"
        i = list(i)
        mycursor.execute(project, i)
        conn.commit()
        print(count)

In [13]:
build_project_table(meta, projects)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217


3. Prior to linking the assay to the tissue, the tissue table must be made using all the tissues and cell types that are present in the manual_tissue_annotation.csv file <br>
The **tissue** table needs:
tissue_name <br>
cell types<br>
healthy or not<br>

In [8]:
manual_tissues = manual_meta[['tissue', 'cell_type', 'status']]
manual_tissues = manual_tissues.drop_duplicates()

In [9]:
def build_tissue_table(tissue_df):
    count = 0
    tissue_tuples = list(tissue_df.to_records(index=False)) #a list of tuples is easily iteratible and easy to store in the database
    for i in tissue_tuples:
        count += 1
        tissue = "INSERT INTO tissue(tissue_name, cell_type, disease_status) VALUES (%s, %s, %s)"
        i = list(i)
        mycursor.execute(tissue, i)
        conn.commit()
    print(count)

In [10]:
build_tissue_table(manual_tissues)

152


4. Load the **modifications table** from the file Tables/modifications.csv into the database

In [11]:
mod_csv = csv.reader(open('../Tables/modifications.csv'), delimiter=';')
for row in mod_csv:
    mod_insert = "INSERT INTO modifications(mod_id, modification_type, mass_difference) VALUES(%s, %s, %s)"
    mycursor.execute(mod_insert, row) 
    conn.commit()
print('Done')

Done


5. Load the **protein** table 

The protein table should be done afterwards based on the proteins found in the assays (merge with uniprot csv file on the uniprotID)

4. The manual_tissue_annotation.csv file will be used to build the **assay table**: <br>
assay_id: auto_incremented<br>
project_id from the project_table, linked to the PXD from the file<br>
filename: assay name<br>
Simultaneously, the link to the tissue must be made<br>

In [12]:
def build_assay_tissue_table(assay_df):
    count = 0
    assay_tuples = list(assay_df.to_records(index=False))
    for i in assay_tuples:
        (accession, filename, pride_tissue, cell_type, tissue, status) = i
        # select project_id
        mycursor.execute("SELECT project_id FROM project WHERE PXD_accession = %s", (accession,))
        projectID_tup = mycursor.fetchone()
        (projectID,) = projectID_tup
        #insert into assay table
        assay = "INSERT INTO assay(project_id, filename) VALUES(%s, %s)"
        projectID_filename = (projectID, filename)
        mycursor.execute(assay, projectID_filename)
        conn.commit()
        #store this automatically generated assay ID for the tissue_to_assay table
        assayID = mycursor.lastrowid
        #select tissue_id
        mycursor.execute("SELECT tissue_id FROM tissue WHERE tissue_name = %s AND cell_type = %s AND disease_status = %s", (tissue, cell_type, status))
        tissueID_tup = mycursor.fetchone()
        (tissueID,) = tissueID_tup
        #insert tissueID and assayID in tissue_to_assay
        tissue_to_assay = "INSERT INTO tissue_to_assay(assay_id, tissue_id) VALUES(%s, %s)"
        assayID_tissueID = (assayID, tissueID)
        mycursor.execute(tissue_to_assay, assayID_tissueID)
        conn.commit()
        count += 1
    print(count)


In [13]:
build_assay_tissue_table(manual_meta)

11860


5. The following step is using the ionbot information to fill in the **assay to peptide**, **peptide** and **peptide_to_modifications** tables.

In [20]:
find_ionbot_files53(projects)

parsed check
HSE3 was stored
parsed check
HSA was stored
parsed check
HSA3 was stored
parsed check
HSE was stored
parsed check
HSB3 was stored
parsed check
HSA2 was stored
parsed check
HSD3 was stored
parsed check
HSC was stored
parsed check
HSC3 was stored
parsed check
HSD2 was stored
parsed check
HSC2 was stored
parsed check
HSB2 was stored
parsed check
HSD was stored
parsed check
HSB was stored
parsed check
HSE2 was stored
parsed check
OR5_120116_MD_PW_LLW1_F13 was stored
parsed check
OR5_111206_MD_LLW3_F17 was stored
parsed check
OR5_111110_MD_LLW2_F21 was stored
parsed check
OR5_111110_MD_LLW2_F19 was stored
parsed check
OR5_111206_MD_LLW3_F14_2 was stored
parsed check
OR5_111110_MD_LLW2_F20 was stored
parsed check
OR5_120116_MD_PW_LLW1_F12 was stored
parsed check
OR5_120116_MD_PW_LLW1_F21 was stored
parsed check
OR5_120116_MD_PW_LLW1_F9_rerun was stored
parsed check
OR5_111110_MD_LLW2_F27 was stored
parsed check
OR5_111110_MD_LLW2_F15 was stored
parsed check
OR5_111206_MD_LLW3_F1

In [9]:
find_ionbot_files54(projects)

parsed check
CHPP_LM3_RP4_1 was stored
parsed check
CHPP_LM3_RP12_2 was stored
parsed check
CHPP_97H_RP17_2 was stored
parsed check
CHPP_97H_RP13_1 was stored
parsed check
CHPP_LM3_RP16_2 was stored
parsed check
CHPP_97H_RP14_1 was stored
parsed check
CHPP_97H_RP10_2 was stored
parsed check
CHPP_97H_RP21_2 was stored
parsed check
CHPP_LM3_RP10_1 was stored
parsed check
CHPP_LM3_RP17_1 was stored
parsed check
CHPP_LM3_RP4_2 was stored
parsed check
CHPP_97H_RP7_2 was stored
parsed check
CHPP_97H_RP23_1 was stored
parsed check
CHPP_LM3_RP22_1 was stored
parsed check
CHPP_97H_RP24_1 was stored
parsed check
CHPP_97H_RP15_1 was stored
parsed check
CHPP_97H_RP7_1 was stored
parsed check
CHPP_97H_RP14_2 was stored
parsed check
CHPP_97H_RP5_2 was stored
parsed check
CHPP_97H_RP19_2 was stored
parsed check
CHPP_97H_RP20_2 was stored
parsed check
CHPP_97H_RP21_1 was stored
parsed check
CHPP_LM3_RP15_1 was stored
parsed check
CHPP_97H_RP3_2 was stored
parsed check
CHPP_LM3_RP20_1 was stored
parsed

/home/compomics/miniconda3/envs/tissuespecific/lib/python3.6/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (5,29) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


parsed check
M530-A10-O291-T113-P7123-1 was stored
parsed check
M529-A10-O290-T113-P7115-1 was stored
parsed check
M530-B06-O291-T113-P7123-1 was stored
parsed check
M528-A09-O289-T113-P7107-1 was stored
parsed check
M530-B01-O291-T113-P7123-1 was stored
parsed check
M510-A08-O251-T113-P6653-1 was stored
parsed check
M532-A06-O293-T113-P7139-1 was stored
parsed check
M529-A09-O290-T113-P7115-1 was stored
parsed check
M533-B01-O294-T113-P7147-1 was stored
parsed check
M529-B10-O290-T113-P7115-1 was stored
parsed check
M529-A05-O290-T113-P7115-1 was stored
parsed check
M529-B04-O290-T113-P7115-1 was stored
parsed check
M510-A10-O251-T113-P6653-1 was stored
parsed check
M531-B04-O292-T113-P7131-1 was stored
parsed check
M530-B07-O291-T113-P7123-1 was stored
parsed check
M532-A01-O293-T113-P7139-1 was stored
parsed check
M531-B02-O292-T113-P7131-1 was stored
parsed check
M531-B07-O292-T113-P7131-1 was stored
parsed check
M510-A06-O251-T113-P6653-1 was stored
parsed check
M510-B03-O251-T113

In [10]:
find_ionbot_files55(projects)

parsed check
Adult_Urinarybladder_bRP_Elite_71 was stored
parsed check
Adult_Testis_bRP_Elite_68 was stored
parsed check
Fetal_Heart_Gel_Velos_21 was stored
parsed check
Adult_Rectum_Gel_Elite_63 was stored
parsed check
Adult_Frontalcortex_bRP_Elite_38 was stored
parsed check
Adult_Platelets_Gel_Elite_48 was stored
parsed check
Adult_Prostate_Gel_Elite_62 was stored
parsed check
Fetal_Heart_bRP_Elite_20 was stored
parsed check
Fetal_Ovary_Gel_Velos_74 was stored
parsed check
Adult_Adrenalgland_bRP_Velos_1 was stored
parsed check
Adult_CD8Tcells_bRP_Velos_43 was stored
parsed check
Adult_Heart_Gel_Velos_7 was stored
parsed check
Adult_Frontalcortex_Gel_Elite_80 was stored
parsed check
Adult_Spinalcord_Gel_Elite_67 was stored
parsed check
Fetal_Ovary_bRP_Velos_41 was stored
parsed check
Adult_Pancreas_bRP_Elite_59 was stored
parsed check
Fetal_Ovary_bRP_Elite_25 was stored
parsed check
Fetal_Placenta_Gel_Velos_14 was stored
parsed check
Fetal_Heart_bRP_Elite_19 was stored
parsed check
Ad

In [4]:
def find_ionbot_files53(projects):
    number_of_files = 0
    for pxd in projects:
        path = '/home/compomics/conode53_pride/PRIDE_DATA/' + str(pxd) + '/IONBOT_v0.6.2/*.mgf.ionbot.csv'
        read_files = []
        for file in glob.glob(path):
            number_of_files += 1
            if file not in read_files:
                read_files.append(file)
                if os.path.getsize(file) != 0:
                    filename = str(file)
                    ionbot_store(file, filename)
    print(number_of_files)

In [5]:
def find_ionbot_files54(projects):
    number_of_files = 0
    for pxd in projects:
        path = '/home/compomics/conode54_pride/PRIDE_DATA/' + str(pxd) + '/IONBOT_v0.6.2/*.mgf.ionbot.csv'
        read_files = []
        for file in glob.glob(path):
            number_of_files += 1
            if file not in read_files:
                read_files.append(file)
                if os.path.getsize(file) != 0:
                    filename = str(file)
                    ionbot_store(file, filename)
    print(number_of_files)

In [6]:
def find_ionbot_files55(projects):
    number_of_files = 0
    for pxd in projects:
        path = '/home/compomics/conode55_pride/PRIDE_DATA/' + str(pxd) + '/IONBOT_v0.6.3/*.mgf.ionbot.csv'
        read_files = []
        for file in glob.glob(path):
            number_of_files += 1
            if file not in read_files:
                read_files.append(file)
                if os.path.getsize(file) != 0:
                    filename = str(file)
                    ionbot_store(file, filename)
    print(number_of_files)

In [7]:
def ionbot_store(file, filename):
    #check if the assay isn't already in the assay table
    filename = filename.split('/')[-1].split('.')[0]

    mycursor.execute("SELECT assay_id FROM assay WHERE filename = %s", (filename,))
    assayIDtup = mycursor.fetchone()
    if assayIDtup is None:
        print('{} is not in assays'.format(filename))
        return
    (assayID,) = assayIDtup
    parser = ionbot_parse(file)
    if parser is False:
        return
    df_validated, spectral_counts = parser

    # use the pandeylines in assay format
    # pandeylines resulted in a pd dataframe with all the proteins and sequences of validated peptides
    # loop over all rows/peptides present in the file (pandey_validated dataframe)
    df_validated_store = df_validated[['proteins', 'matched_peptide', 'modifications']]
    df_validated_tuples = [tuple(x) for x in df_validated_store.to_numpy()]
    for t in df_validated_tuples:
        protID = (t[0])
        pepseq = tuple((t[1],))
        mod = list((t[2],))

        # peptide storage - peptide ID
        sequence = "INSERT INTO peptide(peptide_sequence) VALUES (%s) " \
                    "ON DUPLICATE KEY UPDATE peptide_sequence=peptide_sequence"
        mycursor.execute(sequence, pepseq)
        conn.commit()

        # retrieve peptide_id, do not generate a new id each time!
        mycursor.execute("SELECT peptide_id FROM peptide WHERE peptide_sequence = %s", (pepseq))
        pepIDtup = mycursor.fetchone()
        (pepID,) = pepIDtup

        # link uniProtID = protein in assay to peptide
        proteinID = "INSERT INTO protein(uniprot_id) VALUES (%s) ON DUPLICATE KEY UPDATE uniprot_id=uniprot_id"
        uniprotID = (protID.split('|')[1],)
        mycursor.execute(proteinID, uniprotID)
        conn.commit()

        # relation peptide to protein
        pepToProt = "INSERT INTO peptide_to_protein(uniprot_id, peptide_id) VALUES (%s,%s) " \
                    "ON DUPLICATE KEY UPDATE peptide_id=peptide_id, uniprot_id=uniprot_id"
        uniprotIDstr = ''.join(uniprotID)
        uniprotID_peptideID = (uniprotIDstr, pepID)
        mycursor.execute(pepToProt, uniprotID_peptideID)
        conn.commit()

        for i in mod:
            if pd.isnull(i):
                break
            else:
                # retrieve modification id, peptide id is present
                location = (i.split('|')[0],)
                id = (i[i.find("[")+1:i.find("]")],)

                #retrieve modID
                mycursor.execute("SELECT mod_id FROM modifications WHERE mod_id = %s", (id))
                modIDtup = mycursor.fetchone()
                if modIDtup is None:
                    break
                (modID,) = modIDtup
                # relation peptide_to_modification
                peptoMod = "INSERT INTO peptide_modifications(peptide_id, location, mod_id, assay_id) VALUES (%s, %s, %s, %s)" \
                            "ON DUPLICATE KEY UPDATE peptide_id = peptide_id, mod_id = mod_id, assay_id=assay_id"
                peptoModvalues = pepIDtup + location + modIDtup + assayIDtup
                mycursor.execute(peptoMod, peptoModvalues)
                conn.commit()

        # spectral count for peptide
        count = float('inf')
        for k, v in spectral_counts.items():
            if k == (''.join(pepseq)):
                count = v
                break
        peptideToAssay = "INSERT INTO peptide_to_assay(peptide_id, assay_id, quantification) VALUES (%s, %s, %s) " \
                            "ON DUPLICATE KEY UPDATE peptide_id=peptide_id, assay_id=assay_id"
        peptideID_assayID_count = (pepID, assayID, count)
        mycursor.execute(peptideToAssay, peptideID_assayID_count)
        conn.commit()
    print('{} was stored'.format(filename))

In [8]:
def ionbot_parse(file):
    df = pd.read_csv(file, sep=',')
    # best_psm is equal to 1
    df = df.loc[df['best_psm'] == 1]
    #  q-value-best <= 0.01
    df = df.loc[df['q_value'] <= 0.01]
    # DB column needs to contain 'T' (otherwise decoy hit) +  extra check: only retain swissprot entries (start with sp)
    df = df.loc[df['DB'] == 'T']
    df_validated = df[df['proteins'].astype(str).str.startswith('sp')]
    # remove peptides that are not uniquely identified and are linked to multiple proteins = containing || in proteins
    x = '||'
    # regex is False otherwise it also detects a single | which is in every protein present
    df_validated = df_validated[~df_validated['proteins'].str.contains(x, regex=False)]
    # check not all entries were removed
    if df_validated.empty:
        return False

    # modifications can be linked to unimod id: peptide_modifications: unimod ID vs peptide

    # calculte the spectral counts from each peptide: dict: count
    peptides = df_validated['matched_peptide'].tolist()
    spectral_counts = defaultdict(int)
    for pep in peptides:
        spectral_counts[pep] += 1
    spectral_counts = dict(sorted(spectral_counts.items(), key=lambda item: item[1], reverse=True))
    print('parsed check')
    return df_validated, spectral_counts

# How many peptides were lost due to our filtering?

In [11]:
def ionbot_parse_reduction(file):
    df = pd.read_csv(file, sep=',')
    start_size = df.shape[0]
    # best_psm is equal to 1
    df = df.loc[df['best_psm'] == 1]
    #  q-value-best <= 0.01
    df = df.loc[df['q_value'] <= 0.01]
    # DB column needs to contain 'T' (otherwise decoy hit) +  extra check: only retain swissprot entries (start with sp)
    df = df.loc[df['DB'] == 'T']
    df_validated = df[df['proteins'].astype(str).str.startswith('sp')]
    # remove peptides that are not uniquely identified and are linked to multiple proteins = containing || in proteins
    x = '||'
    # regex is False otherwise it also detects a single | which is in every protein present
    df_validated = df_validated[~df_validated['proteins'].str.contains(x, regex=False)]
    # check not all entries were removed
    if df_validated.empty:
        return False
    end_size = df_validated.shape[0]
    reduction = (end_size/start_size) * 100
    return reduction

In [25]:
reduction_list = []

In [26]:
def find_ionbot_files53_reduc(projects):
    number_of_files = 0
    for pxd in projects:
        path = '/home/compomics/conode53_pride/PRIDE_DATA/' + str(pxd) + '/IONBOT_v0.6.2/*.mgf.ionbot.csv'
        read_files = []
        for file in glob.glob(path):
            number_of_files += 1
            if file not in read_files:
                read_files.append(file)
                if os.path.getsize(file) != 0:
                    filename = str(file)
                    reduction_list.append(ionbot_parse_reduction(file))
    print(len(reduction_list))

In [27]:
def find_ionbot_files54_reduc(projects):
    number_of_files = 0
    for pxd in projects:
        path = '/home/compomics/conode54_pride/PRIDE_DATA/' + str(pxd) + '/IONBOT_v0.6.2/*.mgf.ionbot.csv'
        read_files = []
        for file in glob.glob(path):
            number_of_files += 1
            if file not in read_files:
                read_files.append(file)
                if os.path.getsize(file) != 0:
                    filename = str(file)
                    reduction_list.append(ionbot_parse_reduction(file))
    print(len(reduction_list))

In [28]:
def find_ionbot_files55_reduc(projects):
    number_of_files = 0
    for pxd in projects:
        path = '/home/compomics/conode55_pride/PRIDE_DATA/' + str(pxd) + '/IONBOT_v0.6.3/*.mgf.ionbot.csv'
        read_files = []
        for file in glob.glob(path):
            number_of_files += 1
            if file not in read_files:
                read_files.append(file)
                if os.path.getsize(file) != 0:
                    filename = str(file)
                    reduction_list.append(ionbot_parse_reduction(file))
    print(len(reduction_list))

In [29]:
find_ionbot_files53_reduc(projects)
find_ionbot_files54_reduc(projects)
find_ionbot_files55_reduc(projects)

5040


/home/compomics/miniconda3/envs/tissuespecific/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (5,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


11264
11944


In [30]:
import statistics
statistics.mean(reduction_list)

2.365789134053905